## 範例重點
### 學習在模型開始前檢查各個環節
1. 是否有 GPU 資源
2. 將前處理轉為函式，統一處理訓練、驗證與測試集
3. 將超參數變數化，易於重複使用函式、模型等

In [ ]:
## 確認硬體資源 (如果你是在 Linux, 若是在 Windows, 請參考 https://blog.csdn.net/idwtwt/article/details/78017565)
!nvidia-smi

In [2]:
import os
from tensorflow import keras

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
# 從 Keras 的內建功能中，取得 train 與 test 資料集
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [5]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [7]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


In [8]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [9]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [10]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 6s 120us/sample - loss: 1.9334 - accuracy: 0.3023 - val_loss: 1.7725 - val_accuracy: 0.3699
Epoch 2/100
50000/50000 [==============================] - 6s 114us/sample - loss: 1.7111 - accuracy: 0.3871 - val_loss: 1.7070 - val_accuracy: 0.4005
Epoch 3/100
50000/50000 [==============================] - 5s 109us/sample - loss: 1.6329 - accuracy: 0.4167 - val_loss: 1.6177 - val_accuracy: 0.4264
Epoch 4/100
50000/50000 [==============================] - 5s 106us/sample - loss: 1.5683 - accuracy: 0.4399 - val_loss: 1.6173 - val_accuracy: 0.4225
Epoch 5/100
50000/50000 [==============================] - 6s 113us/sample - loss: 1.5250 - accuracy: 0.4536 - val_loss: 1.5471 - val_accuracy: 0.4490
Epoch 6/100
50000/50000 [==============================] - 5s 107us/sample - loss: 1.4783 - accuracy: 0.4737 - val_loss: 1.5197 - val_accuracy: 0.4589
Epoch 7/100
50000/50000 [===================

50000/50000 [==============================] - 5s 106us/sample - loss: 0.6107 - accuracy: 0.7815 - val_loss: 1.8556 - val_accuracy: 0.5186
Epoch 55/100
50000/50000 [==============================] - 5s 106us/sample - loss: 0.6001 - accuracy: 0.7830 - val_loss: 1.9437 - val_accuracy: 0.5023
Epoch 56/100
50000/50000 [==============================] - 5s 105us/sample - loss: 0.5862 - accuracy: 0.7891 - val_loss: 1.8603 - val_accuracy: 0.5190
Epoch 57/100
50000/50000 [==============================] - 5s 105us/sample - loss: 0.5830 - accuracy: 0.7896 - val_loss: 2.0279 - val_accuracy: 0.5010
Epoch 58/100
50000/50000 [==============================] - 5s 106us/sample - loss: 0.5692 - accuracy: 0.7972 - val_loss: 2.0090 - val_accuracy: 0.5139
Epoch 59/100
50000/50000 [==============================] - 5s 105us/sample - loss: 0.5654 - accuracy: 0.7965 - val_loss: 1.9865 - val_accuracy: 0.5084
Epoch 60/100
50000/50000 [==============================] - 5s 107us/sample - loss: 0.5383 - accuracy

In [11]:
# 訓練模型並檢視驗證集的結果
import matplotlib.pyplot as plt

train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度